# Search Tweets 
este notebook se enfoca en obtener tweets que puedan ser interesantes desde el punto de vista del estudio profesional, se indicarán patrones de busqueda para la descarga de tweets

In [1]:
#Cargamos todas las librerias requeridas
import re
import pandas as pd
import tweepy as twt
from datetime import datetime

In [11]:
#configuramos la conexión al API de twitter mediante tweepy
#tokens de acceso a la api de twitter

ACCESS_TOKEN = '175141573-vHfTt2tErLH9EjCeCwHrR2DXEmLZQncCelLP9axd'
ACCESS_TOKEN_SECRET = 'TCbrWZMl8X7JWcFH0YibD5n6AoOQKNTfxwTtfs8XHSliv'
API_KEY = 'DPrHgZiq5TQrphWcqGqtl06qr'
API_SECRET_KEY = '2vk5ZXhDoSg8itU0nIsOMPMxPKaB2Xk4mdnoqUjCp6dUHPUgAZ'

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAOCScAEAAAAASCUqdpuYquL214J4Jg8OnCmUxXM%3DNUe0ZJ9wZfpCLfNcLHEQoLKycvAowh2cfqvEHOywUTNkCadCC2'

api = twt.Client(bearer_token=BEARER_TOKEN)


In [12]:
def cleantext(text):
    txt_mod = ""
    text = re.sub(r'(https?://)?(www\.)?([a-zA-Z0-9_%]*)\b\.[a-z]{2,4}(\.[a-z]{2})?((/[a-zA-Z0-9_%]*)+)?(\.[a-z]*)?', '', text)
    #text = re.sub(r'@[A-Za-z09]+', '', text)
    #text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/?', '', text)
    text = text.replace('\n', '')
    text = text.replace('.', ' ')
    text = text.replace(',', ' ')
    text = re.sub(r't.co+', '', text)
    
    
    for word in text.split(' '):
        if len(txt_mod) == 0:
            txt_mod = word.lower().strip()
        else:
            txt_mod = txt_mod+' '+word.lower().strip()
                
    return txt_mod

In [4]:
perfil_petro = api.get_user(username='petrogustavo')
perfil_sergio = api.get_user(username='sergio_fajardo')
perfil_fico = api.get_user(username='FicoGutierrez')
perfil_rodolfo = api.get_user(username='ingrodolfohdez')
perfil_quintero = api.get_user(username='QuinteroCalle')

id_perfil_petro = perfil_petro.data.id
id_perfil_sergio = perfil_sergio.data.id
id_perfil_fico = perfil_fico.data.id
id_perfil_rodolfo = perfil_rodolfo.data.id
id_perfil_quintero = perfil_quintero.data.id

In [15]:
response = api.search_recent_tweets(query=query,
                                    tweet_fields = ['id', 
                                                    'text', 
                                                    'author_id', 
                                                    'in_reply_to_user_id',
                                                    'lang',
                                                    'geo',
                                                    'source',
                                                    'public_metrics',
                                                    'created_at'],
                                    max_results = 100)

SyntaxError: invalid syntax (2670545971.py, line 9)

In [23]:
query = '@petrogustavo -is:retweet lang:es'

tweets = twt.Paginator(method=api.search_recent_tweets,
                       query = query,
                       tweet_fields = ['id', 
                                        'text', 
                                        'author_id', 
                                        'in_reply_to_user_id',
                                        'lang',
                                        'geo',
                                        'source',
                                        'public_metrics',
                                        'created_at'],
                       max_results = 100).flatten(3000)


In [27]:
list_tweets = []

for tweet in tweets:
    list_tweets.append([tweet.id,
                        tweet.text,
                        tweet.author_id,
                        tweet.in_reply_to_user_id,
                        tweet.lang,
                        tweet.geo,
                        tweet.source,
                        tweet.public_metrics,
                        tweet.created_at])

In [28]:
df_tweets = pd.DataFrame(list_tweets)

df_tweets.columns = ['id',
                     'tweet',
                     'autor_id',
                     'in_reply_to_user_id',
                     'lang',
                     'geo',
                     'source',
                     'public_metrics',
                     'created_at']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 9 elements

In [29]:
df_tweets.count()

Series([], dtype: int64)

In [36]:
#Ahora vamos a consultar las menciones que ha tenido cada usuario

mentions = twt.Paginator(api.get_users_mentions,
                         id=id_perfil_petro,
                         tweet_fields = ['id', 
                                         'text', 
                                         'author_id', 
                                         'geo',
                                         'source',
                                         'public_metrics',
                                         'created_at'],
                         start_time = datetime.fromisoformat('2022-05-29T19:00:00'),
                         max_results = 100).flatten(10000)

In [37]:
list_mentions = []

for mention in mentions:
    list_mentions.append([mention.id,
                          mention.text,
                          mention.author_id,
                          mention.geo,
                          mention.public_metrics,
                          mention.source,
                          mention.created_at])

In [38]:
df_mentions = pd.DataFrame(list_mentions)
df_mentions.columns = ['id',
                       'tweet',
                       'id_autor',
                       'geolocation',
                       'metrics',
                       'source',
                       'created_at']

df_mentions['tweet'] = df_mentions['tweet'].apply(cleantext)


In [39]:
df_mentions

,id,tweet,id_autor,geolocation,metrics,source,created_at
0,1531649877377265667,@patriciajaniot @ingrodolfohdez @petrogustavo ...,1313198415581413379,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:52:42+00:00
1,1531649872532844544,@falaciadeldia @toyaviudes @petrogustavo 💓💓💓💓,1239137978,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:52:40+00:00
2,1531649867315089410,@las2orillas @ingrodolfohdez @petrogustavo es ...,1351010252,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:52:39+00:00
3,1531649866660823041,@nossiej2419 @elsemblante @petrogustavo @regis...,158510458,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:52:39+00:00
4,1531649861082304513,@toyaviudes @petrogustavo entonces por qué obt...,634594672,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:52:38+00:00
...,...,...,...,...,...,...,...
833,1531646306036371456,@petrogustavo hitler del siglo xxi,891295939595468800,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:38:30+00:00
834,1531646303993749505,la campaña debe continuar debemos redoblar es...,170135191,None,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:38:30+00:00
835,1531646298318950401,@wradiocolombia #micandidatoes el que esta pr...,240471118,None,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,2022-05-31 14:38:28+00:00
836,1531646293986140161,@petrogustavo debe exigir a gobiernos europeos...,574541815,None,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Twitter for Android,2022-05-31 14:38:27+00:00


In [40]:
df_mentions.to_csv('./data/menciones/raw_menciones_petro_31052022.csv', sep='|', index_label='id_df')